In [ ]:
from proseflow.load import load

In [ ]:
import pandas as pd

#TODO ->load()
papers = pd.read_csv("data/pubmed_longevity10k.csv")

In [ ]:
papers

,PAN,NR,TITLE,ABSTRACT
0,1,31806905,Chronic inflammation in the etiology of diseas...,Although intermittent increases in inflammatio...
1,2,31292558,From discoveries in ageing research to therape...,"For several decades, understanding ageing and ..."
2,3,31584784,Platelet-Rich Plasma (PRP): Current Applicatio...,Platelet-rich plasma (PRP) is an autologous se...
3,4,31619788,Regulation of lifespan by neural excitation an...,The mechanisms that extend lifespan in humans ...
4,5,31522175,Metformin and Aging: A Review.,"Metformin is sometimes proposed to be an ""anti..."
...,...,...,...,...
9995,9996,31877416,The oncogenic and druggable hPG80 (Progastrin)...,"BACKGROUND: In colorectal cancer, hPG80 (proga..."
9996,9997,31409905,REX1 promotes EMT-induced cell metastasis by a...,"ZFP42 zinc finger protein (REX1), a pluripoten..."
9997,9998,31356623,Cryptococcus neoformans resists to drastic con...,Metabolically quiescent pathogens can persist ...
9998,9999,31303374,A Single-Cell Transcriptomic Atlas of Human Ne...,"We performed RNA sequencing on 40,000 cells to..."


In [ ]:
!python --version

Python 3.8.2


In [ ]:
import stanza
from spacy_stanza import StanzaLanguage

In [ ]:
import stanza
snlp = stanza.Pipeline('en', package='genia', processors={'ner': 'bionlp13cg'}, use_gpu=False)
# annotate clinical text
doc = snlp('The patient had a sore throat and was treated with Cepacol lozenges.')
# print out all entities
for ent in doc.entities:
    print(f'{ent.text}\t{ent.type}')

2020-12-09 13:07:53 INFO: Loading these models for language: en (English):
| Processor | Package    |
--------------------------
| tokenize  | genia      |
| pos       | genia      |
| lemma     | genia      |
| depparse  | genia      |
| ner       | bionlp13cg |

2020-12-09 13:07:53 INFO: Use device: cpu
2020-12-09 13:07:53 INFO: Loading: tokenize
2020-12-09 13:07:53 INFO: Loading: pos


Exception: Vector file is not provided.

In [ ]:
nlp = load("stanza", use_gpu=False)

2020-12-09 13:06:39 INFO: Loading these models for language: en (English):
| Processor | Package    |
--------------------------
| tokenize  | genia      |
| pos       | genia      |
| lemma     | genia      |
| depparse  | genia      |
| ner       | bionlp13cg |

2020-12-09 13:06:39 INFO: Use device: cpu
2020-12-09 13:06:39 INFO: Loading: tokenize
2020-12-09 13:06:39 INFO: Loading: pos


{'re': 'stanza', 'doc': "Stanford's Stanza Model", 'options': ['stanza1', 'stanza2']} shape {'use_gpu': False}
loading stanza {'lang': 'en', 'package': 'genia', 'processors': {'ner': 'bionlp13cg'}}


2020-12-09 13:06:39 INFO: Downloading these customized packages for language: en (English)...
| Processor       | Package    |
--------------------------------
| tokenize        | genia      |
| pos             | genia      |
| lemma           | genia      |
| depparse        | genia      |
| ner             | bionlp13cg |
| pretrain        | genia      |
| forward_charlm  | pubmed     |
| backward_charlm | pubmed     |

2020-12-09 13:06:39 INFO: File exists: /Users/markus/stanza_resources/en/tokenize/genia.pt.
2020-12-09 13:06:40 INFO: File exists: /Users/markus/stanza_resources/en/pos/genia.pt.
2020-12-09 13:06:40 INFO: File exists: /Users/markus/stanza_resources/en/lemma/genia.pt.
2020-12-09 13:06:40 INFO: File exists: /Users/markus/stanza_resources/en/depparse/genia.pt.


TypeError: int() argument must be a string, a bytes-like object or a number, not 'NoneType'